In [1]:
import math
from igraph import *
import pandas as pd
import collections

df = pd.read_csv('data/anticonflict_study_dataframe.csv')
#df = df.dropna(subset=['TREAT'])
#df.apply(lambda x: x['TREAT'][1])
#print(df['TREAT'])
df = df.assign(TREAT_ID = lambda x: x['TREAT'].str[1])
df['TREAT_ID'] = df['TREAT_ID'].fillna(0)
#df['TREAT_ID']

/var/folders/04/x__0y76s1vd7_tjgq64bshr40000gn/T/ipykernel_90110/27793377.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/anticonflict_study_dataframe.csv')


In [2]:
# Slice the dataframe by school
sch_dfs = []
sch_count = df['SCHID'].max()
for i in range(sch_count):
    sch_dfs.append(df.loc[df['SCHID'] == i+1])
    sch_dfs[i] = sch_dfs[i].reset_index(drop=True)
    print(sch_dfs[i].shape[0])
print(len(sch_dfs))


277
529
138
0
115
467
620
635
394
321
487
565
294
213
103
241
350
446
502
589
444
685
822
839
839
207
289
389
289
558
392
408
384
559
370
603
170
695
445
166
508
441
117
279
440
149
300
319
414
178
551
0
847
0
233
718
279
619
579
661
60


In [3]:
spendtime_noms = ["UID","ST1", "ST2", "ST3", "ST4", "ST5", "ST6", "ST7", "ST8", "ST9", "ST10",'TREAT_ID']
conflict_noms = ["UID","CN1", "CN2", "CN3", "CN4", "CN5",'TREAT_ID']
bestfriend_noms = ["UID","ID","SCHID","BF1", "BF2",'TREAT_ID']


sch_df = sch_dfs[0]
bf = sch_df[sch_df.columns.intersection(bestfriend_noms)]
bf = bf.loc[bf['ID']!=999]
bf['TID'] = bf['UID']-bf['SCHID']*100000
bf.head(20)
#bf[bf.columns] = bf[bf.columns].astype(int)

,SCHID,UID,ID,BF1,BF2,TREAT_ID,TID
0,1,100001,1,140.0,12.0,0,1
1,1,100002,2,21.0,31.0,2,2
2,1,100003,3,999.0,72.0,0,3
3,1,100004,4,19.0,32.0,0,4
4,1,100005,5,258.0,NaN,0,5
5,1,100006,6,13.0,24.0,0,6
6,1,100007,7,39.0,20.0,0,7
7,1,100008,8,22.0,28.0,0,8
8,1,100009,9,21.0,31.0,0,9
9,1,100010,10,247.0,NaN,0,10


In [4]:
def find_tid_index(df,tid):
    # Given TID, return the index in df dataframe
    
    index = df.index[df['TID'] == tid].tolist()[0]
    return index

def build_edge_list(df, nom_list):
    # Given network dataframe, nomination variables list. 
    # Return the vertices index edge list

    edge_list = []
    valid_id = df['TID'].tolist()
    
    for index, row in df.iterrows():
        for nom_name in nom_list:
            if row[nom_name] in valid_id and not math.isnan(row[nom_name]):
                edge_list.append([find_tid_index(df,row['TID']),find_tid_index(df,int(row[nom_name]))])
    
    return edge_list


def add_schl_bf_neighbor(sch_dfs,sch_idx):
    # Calculate 1-jump and 2-jumps number for students in sch_idx school

    bestfriend_noms = ["UID","ID","SCHID","BF1", "BF2",'TREAT_ID']

    sch_df = sch_dfs[sch_idx]
    bf = sch_df[sch_df.columns.intersection(bestfriend_noms)]
    bf = bf.loc[bf['ID']!=999]
    # TID is the true ID for student ID (Entry errors in Student ID)
    bf['TID'] = bf['UID']-bf['SCHID']*100000

    print(bf)

    # Build the edge list. Notice that the vertices in graph is not TID, it should be the vertex index
    nom_list = ["BF1", "BF2"]
    edge_list = build_edge_list(bf,nom_list)


    # Build the undirected graph
    g = Graph(n = bf.shape[0],edges=edge_list,directed=True)
    g.vs['name'] = list(bf['TID'])
    g.vs['tid'] = list(bf['TREAT_ID'])
    g.vs['uid'] = list(bf['UID'])
    g.to_undirected()

    # Calculate the shortest distance from all vertices to the treated vertices 
    target_vertices = g.vs.select(tid_eq="1")
    print(target_vertices.indices)
    vs_distance = g.shortest_paths(source=g.vs, target=target_vertices)

    # Return the number of 1-jump and 2-jumps to the new dataframe

    #print(bf)
    bf1_list = []
    bf2_list = []
    for idx,vertex in enumerate(vs_distance):
        bf1_list.append(collections.Counter(vertex)[1])
        bf2_list.append(collections.Counter(vertex)[2])

    bf['bf_1'] = bf1_list
    bf['bf_2'] = bf2_list

    return bf


    
add_schl_bf_neighbor(sch_dfs,7)

    
    

     SCHID     UID   ID   BF1   BF2 TREAT_ID  TID
0        8  800001    1  22.0  61.0        0    1
1        8  800002    2  54.0  41.0        0    2
2        8  800003    3  55.0   9.0        0    3
3        8  800004    4  60.0   NaN        0    4
4        8  800005    5  71.0   NaN        0    5
..     ...     ...  ...   ...   ...      ...  ...
625      8  800640  640   NaN   NaN        0  640
626      8  800641  641   NaN   NaN        0  641
627      8  800642  642   NaN   NaN        0  642
628      8  800643  643   NaN   NaN        0  643
629      8  800645  645   NaN   NaN        0  645

[630 rows x 7 columns]
[55, 59, 64, 83, 87, 95, 117, 141, 154, 183, 194, 199, 271, 290, 297, 300, 305, 336, 349, 357, 368, 369, 447, 468, 485, 518, 531, 567, 583, 591, 599, 621]


,SCHID,UID,ID,BF1,BF2,TREAT_ID,TID,bf_1,bf_2
0,8,800001,1,22.0,61.0,0,1,0,0
1,8,800002,2,54.0,41.0,0,2,0,0
2,8,800003,3,55.0,9.0,0,3,0,0
3,8,800004,4,60.0,NaN,0,4,1,0
4,8,800005,5,71.0,NaN,0,5,0,0
...,...,...,...,...,...,...,...,...,...
625,8,800640,640,NaN,NaN,0,640,0,0
626,8,800641,641,NaN,NaN,0,641,0,0
627,8,800642,642,NaN,NaN,0,642,0,0
628,8,800643,643,NaN,NaN,0,643,0,0


In [6]:
#def add_bf_neighbor(df):
    